<a href="https://colab.research.google.com/github/buganart/melgan-neurips/blob/master/melgan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@markdown Before starting please save the notebook in your drive by clicking on `File -> Save a copy in drive`

In [3]:
#@markdown Check GPU, should be a Tesla V100
!nvidia-smi -L
import os
print(f"We have {os.cpu_count()} CPU cores.")

GPU 0: Tesla V100-SXM2-16GB (UUID: GPU-242eeefb-9984-6099-c889-1236ed9e5b54)
We have 4 CPU cores.


In [4]:
#@markdown Mount google drive
from google.colab import drive
drive.mount('/content/drive')

from pathlib import Path
if not Path("/content/drive/My Drive/IRCMS_GAN_collaborative_database").exists():
    raise RuntimeError(
        "Shortcut to our shared drive folder doesn't exits.\n\n"
        "\t1. Go to the google drive web UI\n"
        "\t2. Right click shared folder IRCMS_GAN_collaborative_database and click \"Add shortcut to Drive\""
    )

def clear_on_success(msg="Ok!"):
    if _exit_code == 0:
        output.clear()
        print(msg)

Mounted at /content/drive


In [6]:
#@markdown Install wandb and log in
from google.colab import output
!pip install wandb==0.10.8
clear_on_success()
!wandb login
clear_on_success("Ok! Logged into wandb!")

Ok! Logged into wandb!


In [7]:
#@markdown Configuration

#@markdown Directories can be found via file explorer on the left by navigating into `drive` to the desired folders. 
#@markdown Then right-click and `Copy path`.
audio_db_dir = "/content/drive/My Drive/AUDIO DATABASE/RAW Sessions/Roberto Studio Material" #@param {type:"string"}
experiment_dir = "/content/drive/My Drive/IRCMS_GAN_collaborative_database/Experiments/colab-violingan/melgan" #@param {type:"string"}
melgan_output_dir = "/content/drive/My Drive/IRCMS_GAN_collaborative_database/Experiments/colab-violingan/melgan-outputs" #@param {type:"string"}

audio_db_dir = Path(audio_db_dir)
melgan_output_dir = Path(melgan_output_dir)
experiment_dir = Path(experiment_dir)

for path in [experiment_dir, melgan_output_dir]:
    path.mkdir(parents=True, exist_ok=True)

if not audio_db_dir.exists():
    raise RuntimeError("audio_db_dir {audio_db_dir} does not exists.")

In [8]:
!git clone https://github.com/buganart/melgan-neurips

Cloning into 'melgan-neurips'...
remote: Enumerating objects: 29, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 143 (delta 13), reused 14 (delta 5), pack-reused 114
Receiving objects: 100% (143/143), 34.34 MiB | 14.64 MiB/s, done.
Resolving deltas: 100% (65/65), done.


In [9]:
%cd /content/melgan-neurips
%pip install -r requirements.txt
clear_on_success("Dependencies installed.")

Dependencies installed.


In [ ]:
local_wav_dir = Path("/content/wavs/")
!find "{audio_db_dir}" -maxdepth 1 -type f | xargs -t -d "\n" -I'%%' -P 10 -n 1 rsync -a '%%' "$local_wav_dir"/

audio_paths = sorted(list(local_wav_dir.glob("*")))
num_files = len(audio_paths)
print(f"Database has {num_files} files in total.")

rsync -a /content/drive/My Drive/AUDIO DATABASE/RAW Sessions/Roberto Studio Material/consolidado ESAIAS.wav /content/wavs/ 
rsync -a /content/drive/My Drive/AUDIO DATABASE/RAW Sessions/Roberto Studio Material/primer montaje.L.wav /content/wavs/ 
rsync -a /content/drive/My Drive/AUDIO DATABASE/RAW Sessions/Roberto Studio Material/primer montaje.R.1.wav /content/wavs/ 
rsync -a /content/drive/My Drive/AUDIO DATABASE/RAW Sessions/Roberto Studio Material/R121_04.1.wav /content/wavs/ 
rsync -a /content/drive/My Drive/AUDIO DATABASE/RAW Sessions/Roberto Studio Material/ALTEC_04.1.wav /content/wavs/ 
rsync -a /content/drive/My Drive/AUDIO DATABASE/RAW Sessions/Roberto Studio Material/SANKEN_04.1.wav /content/wavs/ 
rsync -a /content/drive/My Drive/AUDIO DATABASE/RAW Sessions/Roberto Studio Material/primer montaje.R.wav /content/wavs/ 
rsync -a /content/drive/My Drive/AUDIO DATABASE/RAW Sessions/Roberto Studio Material/SANKEN_03.1.wav /content/wavs/ 
rsync -a /content/drive/My Drive/AUDIO DATA

In [ ]:
import numpy as np

np.random.seed(123)
test_fraction = 0.1
shuffled_paths = np.random.permutation(audio_paths)
split_at = int((1 - test_fraction) * num_files)
train_paths = shuffled_paths[:split_at]
test_paths = shuffled_paths[split_at:]

with open('train_files.txt', 'w') as f:
    for path in sorted(train_paths):
        print(path, file=f)

with open('test_files.txt', 'w') as f:
    for path in sorted(test_paths):
        print(path, file=f)

print("TRAIN FILES")
!cat train_files.txt

print()
print("TEST FILES")
!cat test_files.txt

In [ ]:
# This done a bit weirdly because setting PYTHONPATH=$PWD removes variables afterwards. A colab bug, maybe.
!env PYTHONPATH="$(pwd)" python scripts/train.py --save_path "$experiment_dir" --data_path . 

In [ ]:
#@markdown Copy the run directory from the beginning of the cell above. 

#@markdown The line looks something like

#@markdown `wandb: Run data is saved locally in /content/drive/.../melgan/wandb/run-20201030_030400-21bhoeww`

#@markdown and we only need the part after the last slash.

run_dir_name = "run-20201030_030400-21bhoeww" #@param {type:"string"}

run_dir = experiment_dir / "wandb" / run_dir_name
export_dir = melgan_output_dir / run_dir_name
export_dir.mkdir(exist_ok=True, parents=True)

melgan_args = run_dir / "files" / "args.yml"
melgan_netG = run_dir / "files" / "best_netG.pt" 

!cp -v "$melgan_args" "$export_dir"
!cp -v "$melgan_netG" "$export_dir"
!cp -v "/content/melgan-neurips/mel2wav/modules.py" "$export_dir"

print()
print(f"Melgan files saved in {export_dir}.")